In [31]:
import xarray as xr
import geopandas as gpd
import rioxarray
from pathlib import Path

In [32]:
def clip_netcdf_by_geojson(netcdf_path, geojson_path, output_path=None, input_crs=None):
    # Load dataset and vector
    ds = xr.open_dataset(netcdf_path)
    gdf = gpd.read_file(geojson_path)

    # 1. Set the correct spatial dims (x/y) and CRS (EPSG:6622)
    ds = ds.rio.set_spatial_dims(x_dim="x", y_dim="y")
    ds = ds.rio.write_crs(input_crs)

    # 2. Reproject GeoDataFrame into the same CRS
    if gdf.crs != ds.rio.crs:
        gdf = gdf.to_crs(ds.rio.crs)

    # 3. Clip by the GeoDataFrame’s geometries
    clipped = ds.rio.clip(gdf.geometry, ds.rio.crs, drop=True)

    # 4. Reproject to EPSG:4326
    clipped_4326 = clipped.rio.reproject("EPSG:4326")

    # 5. (Optional) save to a new NetCDF in 4326
    if output_path:
        clipped_4326.to_netcdf(output_path)

    return clipped_4326

In [38]:
# Example usage:
netcdf_path = r"C:\Users\TyHow\MinersAI Dropbox\Science\Geo Data\Canada\Quebec\processed_data\Quebec_Thorium_75m_QL_F.nc"
geojson_path = r"C:\Users\TyHow\MinersAI Dropbox\Tyler Howe\QuebecProject\geo_data\QuebecROI_2.geojson"

input_nc = Path(netcdf_path)
output_nc = input_nc.with_name(input_nc.stem + "_CLIPPED" + input_nc.suffix)


clipped_ds = clip_netcdf_by_geojson(
    netcdf_path, geojson_path, output_nc, input_crs=6622
)